In [1]:
import numpy as np
import pandas as pd
from sodapy import Socrata

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime as dt

In [10]:
data = pd.read_pickle('Data/timedelta_study.pkl')

In [11]:
data.status.value_counts()

Closed      89470
Pending      2181
Assigned     1065
Open          876
Started        36
Name: status, dtype: int64

In [23]:
data.shape

(93629, 9)

In [7]:
data['closed_date_year'] = data['closed_date'].dt.year
data['closed_date_month'] = data['closed_date'].dt.month
data['closed_date_dayofmonth'] = data['closed_date'].dt.day
data['closed_date_dayofweek'] = data['closed_date'].dt.dayofweek
data['closed_date_hour'] = data['closed_date'].dt.hour

data['created_date_year'] = data['created_date'].dt.year
data['created_date_month'] = data['created_date'].dt.month
data['created_date_dayofmonth'] = data['created_date'].dt.day
data['created_date_dayofweek'] = data['created_date'].dt.dayofweek
data['created_date_hour'] = data['created_date'].dt.hour

data['due_date_year'] = data['due_date'].dt.year
data['due_date_month'] = data['due_date'].dt.month
data['due_date_dayofmonth'] = data['due_date'].dt.day
data['due_date_dayofweek'] = data['due_date'].dt.dayofweek
data['due_date_hour'] = data['due_date'].dt.hour

### Create Timedelta

In [12]:
data['response_time'] = (data['closed_date'] - data['created_date']) / dt.timedelta(minutes=1)
data['remaining_time'] = (data['due_date'] - data['closed_date']) / dt.timedelta(minutes=1)

In [25]:
data[data.status=='Closed'].isnull().sum()/len(data[data.status=='Closed'])*100

unique_key         0.000000
agency             0.000000
complaint_type     0.000000
status             0.000000
closed_date        0.114005
created_date       0.000000
due_date          61.446295
response_time      0.114005
remaining_time    61.454119
dtype: float64

In [29]:
data.drop('due_date', axis=1, inplace=True)
data.drop('remaining_time', axis=1, inplace=True)
data = data.dropna()

### Normalize timedelta by depertment

In [13]:
data.columns

Index(['unique_key', 'agency', 'complaint_type', 'status', 'closed_date',
       'created_date', 'due_date', 'response_time', 'remaining_time'],
      dtype='object')

In [30]:
from sklearn.preprocessing import MinMaxScaler

def scale_timedelta():
    # Scaller type
    scaler = MinMaxScaler()
    
    # dataframe for scaled city data for all cities
    joined = pd.DataFrame()
    
    # Transform data for individual agency
    for agency in data.agency.unique():
        d = data[['agency', 'response_time', 'remaining_time']].loc[data.agency==agency]
        d.drop('agency', axis=1, inplace=True)
        scaler.fit(d)
        transd = scaler.transform(d)
        
        # add scaled date into temporary dataframe
        d['response_time'] = transd[:,0]
        #d['remaining_time'] = transd[:,1]
        
        # join the city data together
        joined = pd.concat([joined, d])
        
    # merge transformed data with entire original dataframe 
    trans_data = data.merge(joined)
        
    # drop original non-scaled features
#     for feature in ['response_time', 'remaining_time']:
#         trans_data.drop(feature, axis=1, inplace=True)

    return trans_data
      
def scale_timedelta():
    # Scaller type
    scaler = MinMaxScaler()
    
    # dataframe for scaled city data for all cities
    joined = pd.DataFrame()
    
    # Transform data for individual agency
    for agency in data.agency.unique():
        d = data[['agency', 'response_time']].loc[data.agency==agency]
        d.drop('agency', axis=1, inplace=True)
        scaler.fit(d)
        transd = scaler.transform(d)
        
        # add scaled date into temporary dataframe
        d['response_time'] = transd[:,0]
        #d['remaining_time'] = transd[:,1]
        
        # join the city data together
        joined = pd.concat([joined, d])
        
    # merge transformed data with entire original dataframe 
    trans_data = data.merge(joined)
        
    # drop original non-scaled features
#     for feature in ['response_time', 'remaining_time']:
#         trans_data.drop(feature, axis=1, inplace=True)

    return trans_data
        

In [31]:
data = scale_timedelta()

### Look at middle 50%

In [32]:
data.columns

Index(['unique_key', 'agency', 'complaint_type', 'status', 'closed_date',
       'created_date', 'response_time'],
      dtype='object')

In [79]:
data[data.agency==any(['DSNY', 'DOT', 'HPD', 'DEP', 'DOB', 'DOHMH'])]

,unique_key,agency,complaint_type,status,closed_date,created_date,response_time


In [59]:
data.agency==any(['DSNY', 'DOT', 'HPD', 'DEP', 'DOB', 'DOHMH'])

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
1499586    False
1499587    False
1499588    False
1499589    False
1499590    False
1499591    False
1499592    False
1499593    False
1499594    False
1499595    False
1499596    False
1499597    False
1499598    False
1499599    False
1499600    False
1499601    False
1499602    False
1499603    False
1499604    False
1499605    False
1499606    False
1499607    False
1499608    False
1499609    False
1499610    False
1499611    False
1499612    False
1499613    Fal

In [60]:
data.agency.value_counts()

DOT      573938
DSNY     423376
DOB      297726
DEP      129528
HPD       44416
DOHMH     30536
DHS          48
TLC          48
Name: agency, dtype: int64

In [76]:
np.where((data.agency=='DOT') | (data.agency=='DSNY') | (data.agency=='HPD') | (data.agency=='DEP') | (data.agency=='DOB') | (data.agency=='DOHMH'))

(array([      0,       1,       2, ..., 1499613, 1499614, 1499615]),)

In [80]:
data.ix((data.agency=='DOT') | (data.agency=='DSNY') | (data.agency=='HPD') | (data.agency=='DEP') | (data.agency=='DOB') | (data.agency=='DOHMH'))

/home/zafrin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [62]:
df = pd.DataFrame()
for agency in data.agency.unique():
    if agency in ['DSNY', 'DOT', 'HPD', 'DEP', 'DOB', 'DOHMH']:
        d = data[data.agency==agency]
    
data[data.agency=='DOT']

,unique_key,agency,complaint_type,status,closed_date,created_date,response_time
694,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
695,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
696,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
697,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
698,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
699,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
700,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
701,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
702,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
703,31754172,DOT,Traffic Signal Condition,Closed,2015-10-14 16:03:00,2015-10-14 16:03:00,0.0
